In [1]:
from functools import reduce
from datetime import datetime

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np


Prueba de concepto

In [2]:
base_url = 'https://www.infocasas.com.uy'
# source_url = 'https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/'
source_url = 'https://www.infocasas.com.uy/alquiler/casas-y-apartamentos/montevideo/'

url_page = source_url + 'pagina{}'.format(100)
print(url_page)
page = urllib.request.urlopen(url_page)
soup = BeautifulSoup(page, 'html.parser')

table = soup.find_all('div', attrs={'class': 'propiedades-slider'})

https://www.infocasas.com.uy/alquiler/casas-y-apartamentos/montevideo/pagina100


In [5]:
neighborhood = [
    [k.text for k in p.find_all('p')] for t in table \
    for p in t.find_all('div')\
    if 'singleLineDots' in p['class']
]
price = [p.text.split()[-1] for t in table \
         for p in t.find_all('div') if 'precio' in p['class']]

desc = [[k.text for k in p.find_all('p')] for t in table \
        for p in t.find_all('div') if 'inDescription' in p['class']]
desc = [k[0] for k in desc]

details = [[d.find_all('span')[0].text for d in p.find_all('div')]\
           for t in table for p in t.find_all('div')\
           if 'contentIcons' in p['class']]
details = pd.DataFrame(details, columns=['rooms', 'bathrooms', 'area_m2'])

data_id = [k.get('data-id', '') for k in table]
data_idproject = [k.get('data-idproyecto', '') for k in table]
link = [base_url + k.find('a')['href'] for k in table]
title = [k.find('a')['title'] for k in table]

proyecto_label = [k.find(class_='proyectoLabel').get_text() if k.find(class_='proyectoLabel') else None for k in table]




df = pd.DataFrame(neighborhood, columns=['neighborhood', 'type'])
df['price'] = price
df['desc'] = desc
df['uris'] = link
df['id'] = data_id
df['idproject'] = data_idproject
df['title'] = title
df['project_label'] = proyecto_label
df = pd.concat([details, df], axis=1)


In [6]:
df

,rooms,bathrooms,area_m2,neighborhood,type,price,desc,uris,id,idproject,title,project_label
0,None,None,None,Parque Rodó,Apartamentos,135.000,TOTALMENTE EQUIPADO! LISTO PARA ENTRAR•\tLivin...,https://www.infocasas.com.uy/proyectos/city-bl...,186062144,2962,City Blanes,Proyecto
1,2,2,107,Malvín,Apartamentos,260.000,"Venta Apartamento En Malvín Sur 2 Dorm, 2 Baño...",https://www.infocasas.com.uy/venta-apartamento...,207359,,"Venta Apartamento En Malvín Sur 2 Dorm, 2 Baño...",None
2,2,1,55,Cordón,Apartamentos,135.650,Apartamento a Estrenar de dos dormitorios al f...,https://www.infocasas.com.uy/venta-2-dormitori...,185853490,,Venta 2 Dormitorios A Estrenar En Cordón Sobre...,None
3,3,1,75,Perez Castellanos,Casas,185.000,Pietrafesa Propiedades vende: Excelente oportu...,https://www.infocasas.com.uy/madreselva-esq-jo...,185987967,,Madreselva Esq. José Pedro Varela,None
4,3,3+,270,Puerto Buceo,Apartamentos,930.000,Penthouse ubicado sobre la Rambla de Pocitos m...,https://www.infocasas.com.uy/74977-apartamento...,185485237,,74977 - Apartamento De 3 Dormitorios En Venta ...,None
5,None,None,None,Barrio Sur,Apartamentos,307.899,ABANA VENDE APARTAMENTO ALTA GAMA 3 DORMTORIOS...,https://www.infocasas.com.uy/unidades/venta/18...,185729634,,Increible. Alta Gama. Totalmente Accesible. Ba...,Proyecto
6,Monoambiente,1,35,Ciudad Vieja,Oficinas,108.000,Excelente oportunidad de inversión en uno de l...,https://www.infocasas.com.uy/ciudad-vieja-ref-...,186050841,,Ciudad Vieja - Ref. 8005c,None
7,2,2,90,Prado,Casas,180.000,Casa al frente a mtrs de importante avenida ¡!...,https://www.infocasas.com.uy/casa-al-frente-a-...,186026018,,Casa Al Frente A Mtrs De Importante Avenida !!,None
8,2,1,77,Pocitos,Apartamentos,222.000,MARINA 26 IIEstrena Marzo 2019. Sobre 26 de M...,https://www.infocasas.com.uy/2-dormitorios-a-p...,185792762,,2 Dormitorios A Pasos De Playa Pocitos,None
9,2,1,58,La Blanqueada,Apartamentos,124.900,LAGÖM HERRERA Muy próximo a Nuevo Centro S...,https://www.infocasas.com.uy/viv-y-disfrut-tu-...,185822805,,Viví Y Disfrutá Tu Mejor Inversión Sobre L. Al...,None


In [4]:
def retrive_data(index,
                 url_base='https://www.infocasas.com.uy',
                 url_filter='/venta/casas-apartamento/montevideo/'):
    url_page = url_base + url_filter + 'pagina{}'.format(index)
    print(url_page)
    try:
        page = urllib.request.urlopen(url_page)
        soup = BeautifulSoup(page, 'html.parser')

        table = soup.find_all('div', attrs={'class': 'propiedades-slider'})
        neighborhood = [
            [k.text for k in p.find_all('p')] for t in table \
            for p in t.find_all('div')\
            if 'singleLineDots' in p['class']
        ]
        price = [p.text.split()[-1] for t in table \
                 for p in t.find_all('div') if 'precio' in p['class']]
        desc = [[k.text for k in p.find_all('p')] for t in table \
                for p in t.find_all('div') if 'inDescription' in p['class']]
        desc = [k[0] for k in desc]
        details = [[d.find_all('span')[0].text for d in p.find_all('div')]\
                   for t in table for p in t.find_all('div')\
                   if 'contentIcons' in p['class']]
        details = pd.DataFrame(details, columns=['rooms', 'bathrooms', 'area_m2'])
        data_id = [k.get('data-id', '') for k in table]
        data_idproject = [k.get('data-idproyecto', '') for k in table]
        link = [url_base + k.find('a')['href'] for k in table]
        proyecto_label = [k.find(class_='proyectoLabel').get_text() if k.find(class_='proyectoLabel') else None for k in table]

        df = pd.DataFrame(neighborhood, columns=['neighborhood', 'type'])
        df['price'] = price
        df['desc'] = desc
        df['uris'] = link
        df['id'] = data_id
        df['idproject'] = data_idproject
        df['project_label'] = proyecto_label
        df['page'] = index
        df = pd.concat([details, df], axis=1)
        
    except ValueError as e:
        df = pd.DataFrame([])
    
    return df

In [5]:
indexs = np.arange(1, 269)
resume = []
for kidx in indexs:
    resume.append(retrive_data(kidx))

https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina1
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina2
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina3
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina4
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina5
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina6
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina7
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina8
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina9
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina10
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina11
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina12
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina13
https://www.infocasas.com.uy/venta/casas-apartamento/montevi

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina19
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina20
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina21
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina22
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina23
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina24
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina25
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina26
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina27
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina28
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina29
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina30
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina31
https://www.infocasas.com.uy/venta/casas-apartament

https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina131
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina132
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina133
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina134
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina135
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina136
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina137
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina138
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina139
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina140
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina141
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina142
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina143
https://www.infocasas.com.uy/venta/cas

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina189
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina190
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina191
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina192
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina193
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina194
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina195
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina196
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina197
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina198
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina199
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina200
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina201
https://www.infocasas.com.uy/venta/cas

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina226
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina227
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina228
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina229
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina230
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina231
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina232
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina233
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina234
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina235


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina236
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina237
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina238
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina239
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina240
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina241
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina242
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina243
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina244
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina245
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina246
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina247
https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina248
https://www.infocasas.com.uy/venta/cas

In [6]:
dataset = reduce(lambda x, y: pd.concat([x,y], axis=0), resume)
dataset.reset_index(inplace=True, drop=True)

In [7]:
len(resume)

268

In [10]:
file_name = 'raw_rent_house_dataset_{}.csv'.format(datetime.today())

In [11]:
dataset.to_csv('../data/{}'.format(file_name), index=False)

In [20]:
area = (dataset['area_m2'].str.replace('.', ''))
area[area == 'N/A'] = None
area = area.astype(float)

In [21]:
price = dataset['price'].str.replace('.', '')
price[price == 'Consultar'] = None
price[price == 'Precio'] = None
price = price.astype(float)

In [22]:
rooms = dataset['rooms'].str.replace('+', '')
rooms[rooms == 'Dormitorios'] = None
rooms[rooms == 'Monoambiente'] = 0
rooms = rooms.astype(float)


In [23]:
bathrooms = dataset['bathrooms'].str.replace('+', '')
bathrooms[bathrooms == 'Baños'] = None
bathrooms = bathrooms.astype(float)

In [24]:
mask = area<5000

In [25]:
dataset.loc[:, 'rooms'] = rooms
dataset.loc[:, 'bathrooms'] = bathrooms
dataset.loc[:, 'area_m2'] = area
dataset.loc[:, 'price'] = price

In [26]:
dataset.to_csv('../data/dataset_houses_{}.csv'.format(datetime.today()), index=False)